In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path = ['../../R98.xlsx', '../../R99.xlsx',
             '../../R1401.xlsx', '../../R1400.xlsx',
             '../../U98.xlsx', '../../U99.xlsx',
             '../../U1401.xlsx', '../../U1400.xlsx']

# Map for accessinng dataFrames of each year easily
file_to_dfs_map = {}

for path in file_path: 
    
    file_identifier = path.split('/')[-1].split('.')[0]
    dataframes_list = []

    excel_file = pd.ExcelFile(path)
    sheetNames = excel_file.sheet_names
    

    for sheet in sheetNames:
        df_name = f'df_{sheet}'  # Create a dynamic DataFrame name
        globals()[df_name] = pd.read_excel(path, sheet_name=sheet)
        dataframes_list.append(df_name)
        print(f'{df_name} DataFrame created with shape: {globals()[df_name].shape}')
    
    file_to_dfs_map[file_identifier] = dataframes_list

df_df_R98Data DataFrame created with shape: (18430, 17)
df_df_R98P1 DataFrame created with shape: (64296, 10)
df_df_R98P2 DataFrame created with shape: (18430, 45)
df_df_R98P3S01 DataFrame created with shape: (516797, 6)
df_df_R98P3S02 DataFrame created with shape: (4175, 6)
df_df_R98P3S03 DataFrame created with shape: (22378, 4)
df_df_R98P3S04 DataFrame created with shape: (68767, 5)
df_df_R98P3S05 DataFrame created with shape: (87170, 4)
df_df_R98P3S06 DataFrame created with shape: (30600, 4)
df_df_R98P3S07 DataFrame created with shape: (35450, 4)
df_df_R98P3S08 DataFrame created with shape: (28627, 4)
df_df_R98P3S09 DataFrame created with shape: (10839, 4)
df_df_R98P3S10 DataFrame created with shape: (0, 4)
df_df_R98P3S11 DataFrame created with shape: (11269, 4)
df_df_R98P3S12 DataFrame created with shape: (67753, 4)
df_df_R98P3S13 DataFrame created with shape: (197127, 3)
df_df_R98P3S14 DataFrame created with shape: (4387, 4)
df_df_R98P4S01 DataFrame created with shape: (10820, 18)

In [13]:
# make new column for sheet 7 part 3 of datas which show the main category of transport cost
for file_name in file_to_dfs_map.keys():
    globals()[file_to_dfs_map[file_name][9]]['code_categorical'] = globals()[file_to_dfs_map[file_name][9]]['code'].astype('str').str[0:3]

In [25]:
#adding label rurall or urban to dataFrame
# 1: Rurall
# 2: urban

for file_name in ['R98', 'R99', 'R1400', 'R1401']:
    globals()[file_to_dfs_map[file_name][9]]['R/U'] = 1

for file_name in ['U98', 'U99', 'U1400', 'U1401']:
    globals()[file_to_dfs_map[file_name][9]]['R/U'] = 0

In [43]:
globals()[file_to_dfs_map['R98'][9]]['code_categorical'].unique()

array(['722', '723', '732', '724', '731', '734', '735', '733'],
      dtype=object)

In [46]:
temp = pd.get_dummies(globals()[file_to_dfs_map['R98'][9]], columns= ['code_categorical'])

In [48]:
temp.columns

Index(['Address', 'code', 'purchased', 'value', 'R/U', 'code_categorical_722',
       'code_categorical_723', 'code_categorical_724', 'code_categorical_731',
       'code_categorical_732', 'code_categorical_733', 'code_categorical_734',
       'code_categorical_735'],
      dtype='object')

In [52]:
temp2 = temp.groupby(by= 'Address').agg({
    'value': 'sum',
    'R/U': 'first',
    'code_categorical_722': 'sum',
    'code_categorical_723': 'sum',
    'code_categorical_724': 'sum',
    'code_categorical_731': 'sum',
    'code_categorical_732': 'sum',
    'code_categorical_733': 'sum',
    'code_categorical_734': 'sum',
    'code_categorical_735': 'sum'
}).reset_index()

In [55]:
# Apply a lambda function to every element in the DataFrame
temp2.iloc[:, 3:] = temp2.iloc[:, 3:].applymap(lambda x: 1 if x > 1 else x)

C:\Users\Poumi008\AppData\Local\Temp\ipykernel_26608\3870261024.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  temp2.iloc[:, 3:] = temp2.iloc[:, 3:].applymap(lambda x: 1 if x > 1 else x)


In [56]:
temp2

,Address,value,R/U,code_categorical_722,code_categorical_723,code_categorical_724,code_categorical_731,code_categorical_732,code_categorical_733,code_categorical_734,code_categorical_735
0,20001383908,750000,1,1,0,0,0,1,0,0,0
1,20001383911,370000,1,0,0,0,0,1,0,0,0
2,20001383915,1000000,1,1,0,0,0,0,0,0,0
3,20001383916,1050000,1,1,0,0,0,0,0,0,0
4,20001383920,350000,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
16776,23006709716,220000,1,0,0,0,0,1,0,0,0
16777,23006709717,180000,1,0,0,0,0,1,0,0,0
16778,23006709720,3030000,1,1,0,0,0,1,0,0,0
16779,23006709723,1250000,1,1,0,0,0,1,0,0,0


In [ ]:
for file_name in file_to_dfs_map.keys():
    dfs= []
    globals()[file_to_dfs_map[file_name][9]] = pd.get_dummies(globals()[file_to_dfs_map[file_name][9]], columns= ['code_categorical'])
    globals()[file_to_dfs_map[file_name][9]] = globals()[file_to_dfs_map[file_name][9]].groupby(by= 'Address').agg({
    'value': 'sum',
    'R/U': 'first',
    'code_categorical_722': 'sum',
    'code_categorical_723': 'sum',
    'code_categorical_724': 'sum',
    'code_categorical_731': 'sum',
    'code_categorical_732': 'sum',
    'code_categorical_733': 'sum',
    'code_categorical_734': 'sum',
    'code_categorical_735': 'sum'
}).reset_index()
    
    dfs.append(globals()[file_to_dfs_map[file_name][9]])  


    # handling p2_ sum of members that use vehicle, motorcycle, bicycle
    temp = globals()[file_to_dfs_map[file_name][2]].groupby(by=['Address']).agg({
    'vehicle': 'sum',
    'motorcycle': 'sum',
    'bicycle': 'sum'
}).reset_index().copy()
    dfs.append(temp)

    

In [60]:
globals()[file_to_dfs_map['R98'][1]].groupby(by=['Address']).agg({
    'vehicle': 'sum',
    'motorcycle': 'sum',
    'bicycle': 'sum'
}).reset_index()

,Address,vehicle,motorcycle,bicycle
0,20001383908,0,1,0
1,20001383911,0,0,0
2,20001383915,1,0,0
3,20001383916,1,0,0
4,20001383920,0,0,0
...,...,...,...,...
18425,23006709717,0,0,1
18426,23006709720,1,0,0
18427,23006709723,0,1,0
18428,23006709807,0,0,0


In [63]:
globals()[file_to_dfs_map['R98'][1]]['gender_encoded'] = globals()[file_to_dfs_map['R98'][1]]['gender'].map({
    'Male': 1,
    'Female': 0
})
globals()[file_to_dfs_map['R98'][1]] = pd.get_dummies(globals()[file_to_dfs_map['R98'][1]], columns= ['occupationalst'])

In [64]:
globals()[file_to_dfs_map['R98'][1]]

,Address,member,relation,gender,age,literacy,studying,degree,maritalst,gender_encoded,occupationalst_Housewife,occupationalst_IncomeWOJob,occupationalst_Other,occupationalst_Student,occupationalst_child,occupationalst_employed,occupationalst_unemployed
0,20010394208,1,Head,Male,50,literate,No,Elemantry,Married,1,False,False,False,False,False,True,False
1,20010394208,2,Spouse,Female,45,illiterate,No,illiterate,Married,0,True,False,False,False,False,False,False
2,20010394208,3,Child,Female,26,literate,No,Elemantry,Married,0,True,False,False,False,False,False,False
3,20010394208,4,Child,Male,25,literate,No,Bachelor,Single,1,False,False,False,False,False,True,False
4,20010394208,5,Child,Male,14,literate,Yes,Secondary,Single,1,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64291,23001701017,3,Child,Male,21,literate,No,College,Single,1,False,False,False,False,False,True,False
64292,23001701020,1,Head,Male,57,literate,No,Secondary,Married,1,False,False,False,False,False,True,False
64293,23001701020,2,Spouse,Female,50,literate,No,Secondary,Married,0,True,False,False,False,False,False,False
64294,23001701020,3,Child,Male,26,literate,Yes,Bachelor,Single,1,False,False,False,True,False,False,False


In [ ]:
globals()[file_to_dfs_map['R98'][1]].groupby(by= ['Address'])